# Readme

Pre-requisites
*   Set up weight and bias account: [Wandb](https://wandb.ai/site/)
*   Upload dataset on drive following the hierarchy below: <br>
<pre>
MyDrive
|_ Final_project
   |_ 2000dataset.zip
</pre>

*   Upload the zip containing the code in drive following the herachy below:
<pre>
MyDrive
|_ Final_project
   |_ RealESRGAN
      |_ Real-ESRGAN.zip
</pre>

*   for **inference** using generator obtained after fine tuning on the dataset, ensure generator is at the hierarchy below:

 <pre>
MyDrive
|_ Final_project
   |_ RealESRGAN
      |_generator_name.pth
</pre>


**Fine-Tuning**: Follow steps 1 to 5 in order. <br> <br>
**Inference**:
*   The real-ESRGAN code and the generator is required for inference.
*   Run step 1 and Step 6. *Note: At step 6,replace the image path with correct image path or folder path.*


#1 Install dependencies

In [ ]:
from google.colab import files, drive
import os
import zipfile
import shutil

In [ ]:
drive.mount('/content/drive', force_remount=True)
code_zip_path = '/content/drive/MyDrive/Final_project/RealESRGAN/Real-ESRGAN.zip'
unzip_path = '/content'
!unzip -q $code_zip_path -d $unzip_path
!ls -l $unzip_path
%cd /content/Real-ESRGAN

Mounted at /content/drive
total 12
drwx------ 5 root root 4096 Sep 23 19:37 drive
drwxrwxrwx 9 root root 4096 Aug 11 07:59 Real-ESRGAN
drwxr-xr-x 1 root root 4096 Sep 19 13:40 sample_data
/content/Real-ESRGAN


In [ ]:
!pip install basicsr
!pip install -r requirements.txt
!python setup.py develop
!pip install wandb -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 28.0 MB/s eta 0:00:00
  Created wheel for basicsr: filename=basicsr-1.4.2-py3-none-any.whl size=214819 sha256=a5a160cb49712364e114010949a7c4f31212b520035480bb09286b722f1faaf5
  Stored in directory: /root/.cache/pip/wheels/9a/e3/e4/58f29bfabb622dd40b6d9839318ce5bf092062b81ca3aa19ea
Successfully built basicsr
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 5.2 MB/s eta 0

In [ ]:
#Corrects import line from degradations.py as this version is obsolete
#https://github.com/xinntao/Real-ESRGAN/issues/801
#Note: if python version was updated on colab, change the file_path to correct one.
file_path = "/usr/local/lib/python3.12/dist-packages/basicsr/data/degradations.py"
line_to_replace = "from torchvision.transforms.functional_tensor import rgb_to_grayscale"
new_line = "from torchvision.transforms.functional import rgb_to_grayscale"

with open(file_path, 'r') as file:
    lines = file.readlines()

lines = [new_line if line.strip() == line_to_replace else line for line in lines]

with open(file_path, 'w') as file:
    file.writelines(lines)

print(f"Replacing line '{line_to_replace}' with '{new_line}' in file {file_path}")

Replacing line 'from torchvision.transforms.functional_tensor import rgb_to_grayscale' with 'from torchvision.transforms.functional import rgb_to_grayscale' in file /usr/local/lib/python3.12/dist-packages/basicsr/data/degradations.py


In [ ]:
# log in to Weights & Biases, not required if doing infernce only
import wandb
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dsha43925 (dsha43925-middlesex-university-mauritius) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

#2 Upload dataset and create meta info

In [ ]:
dataset_zip_path = '/content/drive/MyDrive/Final_project/2000dataset.zip'
!unzip -q $dataset_zip_path -d $unzip_path

In [ ]:
#sometime compressing large folders can create an ini file in a folder. check if the lr hr images differ
hr_dir = "/content/dataset/train/hr"
lr_dir = "/content/dataset/train/lr"

hr_files = sorted(os.listdir(hr_dir))
lr_files = sorted(os.listdir(lr_dir))

print("HR files not in LR:", set(hr_files) - set(lr_files))
print("LR files not in HR:", set(lr_files) - set(hr_files))

HR files not in LR: set()
LR files not in HR: {'desktop.ini'}


In [ ]:
rm "/content/dataset/train/lr/desktop.ini"

In [ ]:
#4 generate the meta info
!python scripts/generate_meta_info_pairdata.py \
    --input /content/dataset/train/hr /content/dataset/train/lr \
    --root /content/dataset /content/dataset \
    --meta_info /content/dataset/meta_info/meta_info_satelliteimages_train.txt

!python scripts/generate_meta_info_pairdata.py \
    --input /content/dataset/val/hr /content/dataset/val/lr \
    --root /content/dataset /content/dataset \
    --meta_info /content/dataset/meta_info/meta_info_satelliteimages_val.txt

train/hr/BayOfBengal1_patch_0.png, train/lr/BayOfBengal1_patch_0.png
train/hr/BayOfBengal1_patch_1.png, train/lr/BayOfBengal1_patch_1.png
train/hr/BayOfBengal1_patch_10.png, train/lr/BayOfBengal1_patch_10.png
train/hr/BayOfBengal1_patch_11.png, train/lr/BayOfBengal1_patch_11.png
train/hr/BayOfBengal1_patch_12.png, train/lr/BayOfBengal1_patch_12.png
train/hr/BayOfBengal1_patch_13.png, train/lr/BayOfBengal1_patch_13.png
train/hr/BayOfBengal1_patch_14.png, train/lr/BayOfBengal1_patch_14.png
train/hr/BayOfBengal1_patch_15.png, train/lr/BayOfBengal1_patch_15.png
train/hr/BayOfBengal1_patch_16.png, train/lr/BayOfBengal1_patch_16.png
train/hr/BayOfBengal1_patch_17.png, train/lr/BayOfBengal1_patch_17.png
train/hr/BayOfBengal1_patch_18.png, train/lr/BayOfBengal1_patch_18.png
train/hr/BayOfBengal1_patch_19.png, train/lr/BayOfBengal1_patch_19.png
train/hr/BayOfBengal1_patch_2.png, train/lr/BayOfBengal1_patch_2.png
train/hr/BayOfBengal1_patch_20.png, train/lr/BayOfBengal1_patch_20.png
train/hr/Bay

#3 Download pre trainedmodel for real esrgan 4xplus and pre trained weights for discriminator

In [ ]:
# 5 Download pretrained models
!mkdir -p experiments/pretrained_models
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.3/RealESRGAN_x4plus_netD.pth -P experiments/pretrained_models


--2025-09-14 12:01:48--  https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/387326890/08f0e941-ebb7-48f0-9d6a-73e87b710e7e?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-09-14T12%3A45%3A46Z&rscd=attachment%3B+filename%3DRealESRGAN_x4plus.pth&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-09-14T11%3A45%3A03Z&ske=2025-09-14T12%3A45%3A46Z&sks=b&skv=2018-11-09&sig=qYaVIJvOk5dkAtibTPGCS4sE3JV8ULc%2FYkPXYr9yRio%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1Nzg1MTYwOCwibmJmIjoxNzU3ODUxMzA4LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJv

#4 Create a configuration file in RealEsrgan/options

In [ ]:
# 6. Create custom training configuration file
config_content = """
# general settings
name: finetune_RealESRGANx4plus_400k_pairdata
model_type: RealESRGANModel
scale: 4
num_gpu: auto
manual_seed: 7251

# USM the ground-truth
l1_gt_usm: True
percep_gt_usm: True
gan_gt_usm: False

high_order_degradation: False # do not use the high-order degradation generation process

# dataset and data loader settings
datasets:
  train:
    name: satellite_dataset_train
    type: RealESRGANPairedDataset
    dataroot_gt: /content/dataset
    dataroot_lq: /content/dataset
    meta_info: /content/dataset/meta_info/meta_info_satelliteimages_train.txt
    filename_tmpl: '{}'
    io_backend:
      type: disk

    gt_size: 128
    use_hflip: True
    use_rot: True

    # data loader
    use_shuffle: true
    num_worker_per_gpu: 1
    batch_size_per_gpu: 2
    dataset_enlarge_ratio: 2
    prefetch_mode: ~

  val:
    name: satellite_dataset_val
    type: RealESRGANPairedDataset
    dataroot_gt: /content/dataset
    dataroot_lq: /content/dataset
    meta_info: /content/dataset/meta_info/meta_info_satelliteimages_val.txt
    #filename_tmpl: '{}'
    io_backend:
      type: disk

    gt_size: 256
    use_hflip: False
    use_rot: False

# network structures
network_g:
  type: RRDBNet
  num_in_ch: 3
  num_out_ch: 3
  num_feat: 64
  num_block: 23
  num_grow_ch: 32

network_d:
  type: UNetDiscriminatorSN
  num_in_ch: 3
  num_feat: 64
  skip_connection: True

# path
path:
  # use the pre-trained Real-ESRGAN model
  pretrain_network_g: experiments/pretrained_models/RealESRGAN_x4plus.pth
  param_key_g: params_ema
  strict_load_g: true
  #pretrain_network_d: experiments/pretrained_models/RealESRGAN_x4plus_netD.pth
  #param_key_d: params
  #strict_load_d: true
  #resume_state: ~

# training settings
train:
  ema_decay: 0.999
  optim_g:
    type: Adam
    lr: !!float 2e-5
    weight_decay: 0
    betas: [0.9, 0.99]
  optim_d:
    type: Adam
    lr: !!float 2e-5
    weight_decay: 0
    betas: [0.9, 0.99]

  scheduler:
    type: MultiStepLR
    milestones: [2000]
    gamma: 0.5

  total_iter: 5000
  warmup_iter: -1

  # losses
  pixel_opt:
    type: L1Loss
    loss_weight: 1.0
    reduction: mean
  # perceptual loss
  perceptual_opt:
    type: PerceptualLoss
    layer_weights:
      # before relu
      'conv1_2': 1
      'conv2_2': 1
      'conv3_4': 1
      'conv4_4': 1
      'conv5_4': 1
    vgg_type: vgg19
    use_input_norm: true
    perceptual_weight: 0.05
    style_weight: 0
    range_norm: false
    criterion: l1
  # gan loss
  gan_opt:
    type: GANLoss
    gan_type: vanilla
    real_label_val: 1.0
    fake_label_val: 0.0
    loss_weight: !!float 5e-2

  net_d_iters: 1
  net_d_init_iters: 0

# validation settings
val:
  val_freq: 100
  save_img: True

  metrics:
    psnr:
      type: calculate_psnr
      crop_border: 4
      test_y_channel: false
    ssim:
      type: calculate_ssim
      crop_border: 4
      test_y_channel: false
    niqe:
      type: calculate_niqe
      crop_border: 4
      input_order: 'HWC'
      convert_to: 'y'

# logging settings
logger:
  print_freq: 100
  save_checkpoint_freq: 500
  use_tb_logger: true
  wandb:
    project: real-esrgan-satellite-training
    resume_id: ~
    name: finetune_satelliteimages_experiment
    tags: ['real-esrgan', 'satellite', 'super-resolution']
    notes: "Fine-tuning Real-ESRGAN on satellite imagery dataset"

# dist training settings
dist_params:
  backend: nccl
  port: 29500

"""

# Write the configuration file
with open('options/finetune_satelliteimages.yml', 'w') as f:
    f.write(config_content)

print("Configuration file created: options/finetune_satelliteimages.yml")


Configuration file created: options/finetune_satelliteimages.yml


#5 fine tune model

In [ ]:
#uncomment auto resume if want to fine tune from last checkpoint
print("Starting fine-tuning!")
%cd /content/Real-ESRGAN
!python -m realesrgan.train -opt options/finetune_satelliteimages.yml #--auto_resume

Starting fine-tuning!
/content/Real-ESRGAN
Disable distributed.
2025-09-11 15:52:56,538 INFO: 
                ____                _       _____  ____
               / __ ) ____ _ _____ (_)_____/ ___/ / __ \
              / __  |/ __ `// ___// // ___/\__ \ / /_/ /
             / /_/ // /_/ /(__  )/ // /__ ___/ // _, _/
            /_____/ \__,_//____//_/ \___//____//_/ |_|
     ______                   __   __                 __      __
    / ____/____   ____   ____/ /  / /   __  __ _____ / /__   / /
   / / __ / __ \ / __ \ / __  /  / /   / / / // ___// //_/  / /
  / /_/ // /_/ // /_/ // /_/ /  / /___/ /_/ // /__ / /<    /_/
  \____/ \____/ \____/ \____/  /_____/\____/ \___//_/|_|  (_)
    
Version Information: 
	BasicSR: 1.4.2
	PyTorch: 2.8.0+cu126
	TorchVision: 0.23.0+cu126
2025-09-11 15:52:56,538 INFO: 
  name: finetune_RealESRGANx4plus_400k_pairdata
  model_type: RealESRGANModel
  scale: 4
  num_gpu: 1
  manual_seed: 7251
  l1_gt_usm: True
  percep_gt_usm: True
  gan_gt_usm: False


#6 Inference

In [ ]:
#Upload the zip file containing low resolution images for inference.
#code_zip_path = '/content/drive/MyDrive/Final_project/testImages.zip'
code_zip_path = '/content/drive/MyDrive/Final_project/LRInferenceImages.zip'
unzip_path = '/content'
!unzip -q $code_zip_path -d $unzip_path
!ls -l $unzip_path

total 16
drwx------  5 root root 4096 Sep 23 19:37 drive
drwxrwxrwx  2 root root 4096 Sep 23 18:42 LRInferenceImages
drwxrwxrwx 10 root root 4096 Sep 23 19:38 Real-ESRGAN
drwxr-xr-x  1 root root 4096 Sep 19 13:40 sample_data


##6.1 Inference on any lr image folder

In [ ]:
#replace the image path with the low resolution image to be used for inference.
#replace model path with either the generator which was already fine-tuned
#or with a generator at a new checkpoint
#checkpoint located at /content/Real-ESRGAN/experiments/finetune_RealESRGANx4plus_400k_pairdata/models/net_g_<num>.pth
image_path = "/content/LRInferenceImages"
model_path = "/content/drive/MyDrive/Final_project/RealESRGAN/net_g_2500.pth"

In [ ]:
#output image is at /content/Real-ESRGAN/results
!python /content/Real-ESRGAN/inference_realesrgan.py -n RealESRGAN_x4plus -i {image_path} --outscale 4 --model_path {model_path} --tile 256

# Download the images
shutil.make_archive("/content/results", 'zip', "/content/Real-ESRGAN/results")
files.download("/content/results.zip")

Testing 0 AirbusDhirubhai_AmbaniSolarPlant
	Tile 1/2
	Tile 2/2
Testing 1 AirbusRotterdamPort
	Tile 1/4
	Tile 2/4
	Tile 3/4
	Tile 4/4
Testing 2 AirbusSantiagoAirport
	Tile 1/4
	Tile 2/4
	Tile 3/4
	Tile 4/4
Testing 3 MaxarIndonesia
	Tile 1/4
	Tile 2/4
	Tile 3/4
	Tile 4/4
Testing 4 MaxarYellowStone
	Tile 1/6
	Tile 2/6
	Tile 3/6
	Tile 4/6
	Tile 5/6
	Tile 6/6
Testing 5 TurkeyEarthquake
	Tile 1/4
	Tile 2/4
	Tile 3/4
	Tile 4/4
Testing 6 italystadiumAirbus
	Tile 1/4
	Tile 2/4
	Tile 3/4
	Tile 4/4
Testing 7 pittsburg
	Tile 1/4
	Tile 2/4
	Tile 3/4
	Tile 4/4
Testing 8 plantation
	Tile 1/2
	Tile 2/2


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 6.2 inference with test images at testImages.zip

In [ ]:
test_image_path = "/content/testImages"
folders = os.listdir(test_image_path)
model_path = "/content/drive/MyDrive/Final_project/RealESRGAN/net_g_5000.pth"

for folder in folders:
    image_path = os.path.join(test_image_path, folder)
    if os.path.isdir(image_path):
        print(f"Processing {folder}")
        output_folder = f"/content/Real-ESRGAN/results/{folder}"
        os.makedirs(output_folder, exist_ok=True)

        !python /content/Real-ESRGAN/inference_realesrgan.py -n RealESRGAN_x4plus -i "{image_path}" --outscale 4 --model_path "{model_path}" --output "{output_folder}"

        if os.listdir(output_folder):
            shutil.make_archive(f"/content/results_{folder}", 'zip', output_folder)
            files.download(f"/content/results_{folder}.zip")

Processing desert
Testing 0 desert1_patch_0
Testing 1 desert1_patch_1
Testing 2 desert1_patch_10
Testing 3 desert1_patch_11
Testing 4 desert1_patch_12
Testing 5 desert1_patch_13
Testing 6 desert1_patch_14
Testing 7 desert1_patch_15
Testing 8 desert1_patch_16
Testing 9 desert1_patch_17
Testing 10 desert1_patch_18
Testing 11 desert1_patch_19
Testing 12 desert1_patch_2
Testing 13 desert1_patch_3
Testing 14 desert1_patch_4
Testing 15 desert1_patch_5
Testing 16 desert1_patch_6
Testing 17 desert1_patch_7
Testing 18 desert1_patch_8
Testing 19 desert1_patch_9
Testing 20 desert2_patch_0
Testing 21 desert2_patch_1
Testing 22 desert2_patch_10
Testing 23 desert2_patch_11
Testing 24 desert2_patch_12
Testing 25 desert2_patch_13
Testing 26 desert2_patch_14
Testing 27 desert2_patch_15
Testing 28 desert2_patch_16
Testing 29 desert2_patch_17
Testing 30 desert2_patch_18
Testing 31 desert2_patch_19
Testing 32 desert2_patch_2
Testing 33 desert2_patch_20
Testing 34 desert2_patch_21
Testing 35 desert2_patch_

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Processing snowregion
Testing 0 snowregion1_patch_0
Testing 1 snowregion1_patch_1
Testing 2 snowregion1_patch_10
Testing 3 snowregion1_patch_11
Testing 4 snowregion1_patch_12
Testing 5 snowregion1_patch_13
Testing 6 snowregion1_patch_14
Testing 7 snowregion1_patch_15
Testing 8 snowregion1_patch_16
Testing 9 snowregion1_patch_17
Testing 10 snowregion1_patch_18
Testing 11 snowregion1_patch_19
Testing 12 snowregion1_patch_2
Testing 13 snowregion1_patch_20
Testing 14 snowregion1_patch_21
Testing 15 snowregion1_patch_22
Testing 16 snowregion1_patch_23
Testing 17 snowregion1_patch_24
Testing 18 snowregion1_patch_25
Testing 19 snowregion1_patch_26
Testing 20 snowregion1_patch_27
Testing 21 snowregion1_patch_3
Testing 22 snowregion1_patch_4
Testing 23 snowregion1_patch_5
Testing 24 snowregion1_patch_6
Testing 25 snowregion1_patch_7
Testing 26 snowregion1_patch_8
Testing 27 snowregion1_patch_9
Testing 28 snowregion2_patch_0
Testing 29 snowregion2_patch_1
Testing 30 snowregion2_patch_10
Testing 

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Processing water
Testing 0 water1_patch_0
Testing 1 water1_patch_1
Testing 2 water1_patch_10
Testing 3 water1_patch_11
Testing 4 water1_patch_12
Testing 5 water1_patch_13
Testing 6 water1_patch_14
Testing 7 water1_patch_16
Testing 8 water1_patch_17
Testing 9 water1_patch_18
Testing 10 water1_patch_19
Testing 11 water1_patch_5
Testing 12 water1_patch_6
Testing 13 water1_patch_7
Testing 14 water1_patch_8
Testing 15 water2_patch_3
Testing 16 water2_patch_5
Testing 17 water2_patch_6
Testing 18 water2_patch_7
Testing 19 water3_patch_0
Testing 20 water3_patch_1
Testing 21 water3_patch_10
Testing 22 water3_patch_11
Testing 23 water3_patch_12
Testing 24 water3_patch_15
Testing 25 water3_patch_2
Testing 26 water3_patch_6
Testing 27 water3_patch_8
Testing 28 water3_patch_9
Testing 29 water4_patch_0
Testing 30 water4_patch_1
Testing 31 water4_patch_13
Testing 32 water4_patch_14
Testing 33 water4_patch_2
Testing 34 water4_patch_6
Testing 35 water4_patch_9
Testing 36 water5_patch_0
Testing 37 water

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>